# The python program of anonymizing address data(string data)

## The main idea of how the coding works

1. The Japanese address data is change in to a latin alphabet (60letters at maximum)
  
  
2. Then it is changed to a binary format  
  
  
3. The binary format is divided into 10 groups(each group contains 6letters = 8bits)  
  
  
4. Each binary format in the group is changed into a float point value  
  
  
5. Then for each id we make a dictionary data which is something like below  
  
>{'address': '小田原市田島１９４番地の１６',  
 'address_english': 'odawaraichidashima194banchino16',  
 'address_binary': '011011110110010001100001011101110110000101110010011000010110100101100011011010000110100101100100011000010111001101101000011010010110110101100001001100010011100100110100011000100110000101101110011000110110100001101001011011100110111100110001001101100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_0': '0111111111011011110110010001100001011101110110000101110010011011',  
 'address_float_value_0': 7.822220133298268e+307,  
 'address_float_1': '0111111111011000010110000000000000000000000000000000000000000000',  
 'address_float_value_1': 0.0,  
 'address_float_2': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_2': 0.0,  
 'address_float_3': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_3': 0.0,  
 'address_float_4': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_4': 0.0,  
 'address_float_5': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_5': 0.0,  
 'address_float_6': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_6': 0.0,  
 'address_float_7': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_7': 0.0,  
 'address_float_8': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_8': 0.0,  
 'address_float_9': '0000000000000000000000000000000000000000000000000000000000000000',  
 'address_float_value_9': 0.0,  
 'group': [0, 7, 0, 0, 0, 0, 0, 0, 0, 0]}  
 
 
6. We look at the element from address_float_0 and group out the data which has the same value and insert the group number in the group element inside the dictionary data 


7. See if there is at least k number of data in each group. If this is satisfy we move on to 12.


8. If 7 was not satisfied we put the data that has less than k into another group which has the closest float value


9. After that we look at the groups which was fused and see how much bits are needed to be deleted to have the same value and convert the bits into 0


10. The other elements after the address_float_0 is converted to 0 if the group was changed


11. We change the address_float_0 and group element inside the dictionary data according to 7~10


12. We do 6~10 in the order of address_float_1 to address_float_9
    - From address float_1 to address_float_9 to connect the groups the the elements in the group has to be all the same except for the current group number you added 

## importing libraries

In [1]:
#importing libralies that are needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pykakasi import kakasi
import mojimoji
import struct
from collections import Counter
import time
import copy

## defining the function

### functions neccesary for the conversion

In [2]:
#defining the functions that will be used in this program

#changing the numbers into double precision
def binary(num):
    return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!d', num))

#changing the double float in to a number
def float_to_num(target):
    for i in range(0,len(target),8):
        if i == 0:
            tmp = struct.pack("!B",int(target[i:i+8],2))
        else:
            tmp = tmp + struct.pack("!B",int(target[i:i+8],2))
    return np.float64(struct.unpack("!d",tmp)[0])

### functions that are neccesary for the anonymization

In [3]:
def make_not_anonymized_list_initial(data_dict,data,k):
    tmp = data["address_float_0"].unique()
    not_anonymized_list = []
    group_list = []
    for x,tm in enumerate(tmp):
        a = 0
        group = []
        for i in data_dict:
            if tm == data_dict[i]["address_float_0"]:
                data_dict[i]["group"] = [x]
                group.append(i)
                a += 1
        group_list.append(group)
        if 0<a<k:
            not_anonymized_list.append(group)
    print(len(not_anonymized_list))
    return data_dict

In [4]:
def make_group_list(data_dict,num):
    unique_float_list = []
    for i in data_dict:
        unique_float_list.append(data_dict[i]["address_float_{}".format(num)])
    unique_float_list = np.unique(np.array(unique_float_list))
    group_list_tmp = [data_dict[i]["group"] for i in range(len(data_dict))]
    address_float_list = [data_dict[i]["address_float_{}".format(num)] for i in range(len(data_dict))]
    for x,unique in enumerate(unique_float_list):
        for i in data_dict:
            if unique == address_float_list[i]:
                group_list_tmp[i].append(x)
    return group_list_tmp

In [5]:
def make_not_anonymized_and_group_list(data_dict,group_list_tmp,k):
    unique_group_list = []
    for i in data_dict:
        if data_dict[i]["group"] not in unique_group_list:
            unique_group_list.append(copy.deepcopy(data_dict[i]["group"]))
    group_list = []
    not_anonymized_list = []
    for unique in unique_group_list:
        group = []
        for x,tmp in enumerate(group_list_tmp):
            if tmp == unique:
                group.append(x)
        group_list.append(group)
        if 0 < len(group) < k:
            not_anonymized_list.append(group)
    return [group_list, not_anonymized_list]

In [6]:
def make_anonymized_group_list(group_list,not_anonymized_list,data_dict,k,num):
    start = time.time()
    while len(not_anonymized_list) != 0:
        for not_anonymized in not_anonymized_list:
            diff = 0
            for x,i in enumerate(group_list):
                if data_dict[not_anonymized[0]]["group"] != data_dict[i[0]]["group"] and data_dict[not_anonymized[0]]["group"][:-1] == data_dict[i[0]]["group"][:-1]:
                    if diff == 0:
                        diff = abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)])
                        closest_group = data_dict[i[0]]["group"][-1]
                        closest_group_num = x
                    elif diff > abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)]):
                        diff = abs(data_dict[not_anonymized[0]]["address_float_value_{}".format(num)]-data_dict[i[0]]["address_float_value_{}".format(num)])
                        closest_group = data_dict[i[0]]["group"][-1]
                        closest_group_num = x
            for j in not_anonymized:
                group_list[closest_group_num].append(j)
                data_dict[j]["group"][-1] = closest_group
            group_list.remove(not_anonymized)
            break
        not_anonymized_list = []
        for group in group_list:
            if 0 < len(group) < k:
                not_anonymized_list.append(group)
    return group_list,data_dict

In [7]:
def make_anonymized_data_dict(group_list,data_dict,precision_num,num_1):
    for group in group_list:
        anonymized_data_list = []
        for x,i in enumerate(group):
            tmp = data_dict[i]["address_float_{}".format(num_1)]
            if x == 0:
                fix = data_dict[i]["address_float_{}".format(num_1)]
            elif fix != tmp:
                for k in range(1,precision_num):
                    if fix[0:-k] == tmp[0:-k]:
                        anonymized_data_list.append(fix[0:-k] + "".join("0" for l in range(k)))
                        break
        if len(anonymized_data_list)>0:
            for x,anony in enumerate(anonymized_data_list):
                if x == 0:
                    anonymized_data = anony
                elif anonymized_data != anony:
                    for k in range(1,precision_num):
                        if anonymized_data[0:-k] == anony[0:-k]:
                            anonymized_data = (anonymized_data[0:-k] + "".join("0" for l in range(k)))
                            break
            key_list = list(data_dict[0].keys())
            key_num = key_list.index("address_float_{}".format(num_1))
            for x,i in enumerate(group):
                data_dict[i]["address_float_{}".format(num_1)] = anonymized_data
                for num in range(key_num+1,len(key_list)-1):
                    if num%2 == 1:
                        data_dict[i][key_list[num]] = "".join("0" for l in range(precision_num))
                    else:
                        data_dict[i][key_list[num]] = np.float64(0)
    return data_dict

## Making the data 

In [8]:
#initial coding to convert the japanese character into a latin alphabet
kakasi = kakasi()
kakasi.setMode('H', 'a')
kakasi.setMode('K', 'a')
kakasi.setMode('J', 'a')
conv = kakasi.getConverter()

In [9]:
#Gettng the data from the csv file and deleting the columns that are not needed
data_address_company_original = pd.read_csv("/home/harada/Documents/data/address_data/14_kanagawa_all_20180928.csv",encoding="SHIFT-JIS", header=None)
columns_list = [0,1,2,3,4,5,7,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
data_address_company_original = data_address_company_original.drop(columns=columns_list)
data_address_company_original.columns = ["company_name","company_group_id","address_1","address_2","address_3"]
data_address_company_original.head()

,company_name,company_group_id,address_1,address_2,address_3
0,藤沢簡易裁判所,101,神奈川県,藤沢市,朝日町１−８
1,神奈川県,201,神奈川県,横浜市中区,日本大通１
2,相模原市,201,神奈川県,相模原市中央区,中央２丁目１１−１５
3,小田原市,201,神奈川県,小田原市,荻窪３００
4,茅ヶ崎市,201,神奈川県,茅ヶ崎市,茅ヶ崎１丁目１−１


In [10]:
#Coping the dataset into another variable
data_address_company = copy.deepcopy(data_address_company_original)
data_address_company = data_address_company.drop(["address_1"],axis = 1)

In [11]:
#making the columns which concats the address data
#makes the data into a latin alphabetical letter
#the latin alphabet is changed into the binary format
address_list = []
for i in range(len(data_address_company)):
    address_list.append(data_address_company.address_2[i] + data_address_company.address_3[i])
address_list = np.array(address_list)
data_address_company["address"] = address_list
address_english = []
for i in data_address_company.address:
    address_english.append(conv.do(mojimoji.zen_to_han(i))[:60])
address_english = np.array(address_english)
data_address_company["address_english"] = address_english
address_binary = []
for i in data_address_company["address_english"]:
    address_binary.append("".join(format(ord(j),"b").rjust(8,"0") for j in i).ljust(480,"0"))
address_binary = np.array(address_binary)
data_address_company["address_binary"] = address_binary
data_address_company = data_address_company.reset_index(drop = True)
data_address_company.head()

,company_name,company_group_id,address_2,address_3,address,address_english,address_binary
0,藤沢簡易裁判所,101,藤沢市,朝日町１−８,藤沢市朝日町１−８,fujisawashiasahimachi1−8,0110011001110101011010100110100101110011011000...
1,神奈川県,201,横浜市中区,日本大通１,横浜市中区日本大通１,yokohamashinakakunihondaitoori1,0111100101101111011010110110111101101000011000...
2,相模原市,201,相模原市中央区,中央２丁目１１−１５,相模原市中央区中央２丁目１１−１５,sagamiharashichuuoukuchuuou2choume11−15,0111001101100001011001110110000101101101011010...
3,小田原市,201,小田原市,荻窪３００,小田原市荻窪３００,odawarashiogikubo300,0110111101100100011000010111011101100001011100...
4,茅ヶ崎市,201,茅ヶ崎市,茅ヶ崎１丁目１−１,茅ヶ崎市茅ヶ崎１丁目１−１,chigasakishichigasaki1choume1−1,0110001101101000011010010110011101100001011100...


In [12]:
#making the backup of the dataset
data_address_company_backup = copy.deepcopy(data_address_company)

In [13]:
#dividing the binary into 48bits each and changing the 48bits into a float64 value
address_float_value = []
address_float = []
for j in range(0,len(data_address_company.address_binary[0]),48):
    tmp_float = []
    tmp_float_value = []
    for i,data in enumerate(data_address_company.address_binary):
        tmp_float.append(binary(np.float64(int(data[j:j+48],2)/(2**48-1)*1.7976931348623157e+308)))
        tmp_float_value.append(np.float64(int(data[j:j+48],2)/(2**48-1)*1.7976931348623157e+308))
    address_float.append(tmp_float)
    address_float_value.append(tmp_float_value)
for i in range(len(address_float)):
    data_address_company["address_float_{}".format(i)] = address_float[i]
    data_address_company["address_float_value_{}".format(i)] = address_float_value[i]
print(data_address_company.columns)
data_address_company = data_address_company.sort_values("address_float_value_1").reset_index(drop = True)
data_address_company.head()

Index(['company_name', 'company_group_id', 'address_2', 'address_3', 'address',
       'address_english', 'address_binary', 'address_float_0',
       'address_float_value_0', 'address_float_1', 'address_float_value_1',
       'address_float_2', 'address_float_value_2', 'address_float_3',
       'address_float_value_3', 'address_float_4', 'address_float_value_4',
       'address_float_5', 'address_float_value_5', 'address_float_6',
       'address_float_value_6', 'address_float_7', 'address_float_value_7',
       'address_float_8', 'address_float_value_8', 'address_float_9',
       'address_float_value_9'],
      dtype='object')


,company_name,company_group_id,address_2,address_3,address,address_english,address_binary,address_float_0,address_float_value_0,address_float_1,...,address_float_5,address_float_value_5,address_float_6,address_float_value_6,address_float_7,address_float_value_7,address_float_8,address_float_value_8,address_float_9,address_float_value_9
0,優幸不動産株式会社,301,小田原市,田島１２０４番地,小田原市田島１２０４番地,odawaraichidashima1204banchi,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
1,三光興業有限会社,302,小田原市,田島１３５番地,小田原市田島１３５番地,odawaraichidashima135banchi,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
2,株式会社シモカワ,301,小田原市,田島９７２番地の１,小田原市田島９７２番地の１,odawaraichidashima972banchino1,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
3,ｅライフ株式会社,301,小田原市,田島２５６番地６,小田原市田島２５６番地６,odawaraichidashima256banchi6,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0
4,有限会社石井精工,302,小田原市,田島６２３番地,小田原市田島６２３番地,odawaraichidashima623banchi,0110111101100100011000010111011101100001011100...,0111111111011011110110010001100001011101110110...,7.822220e+307,0111111111011000010110100101100011011010000110...,...,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0,0000000000000000000000000000000000000000000000...,0.0


## Anonymizing the dataset (k=20)

In [14]:
#randomly sample 20000 data inside the dataset
#making the backup of the data
data_address_company_for_use = copy.deepcopy(data_address_company.sample(20000).sort_index().reset_index(drop = True))
while (data_address_company_for_use.address_float_0.value_counts() < 20).any():
    data_address_company_for_use = data_address_company.sample(20000).sort_index().reset_index(drop = True)
data_address_company_dict_backup = copy.deepcopy(data_address_company_for_use.drop(data_address_company.columns[0:4],axis = 1).T.to_dict())
data_address_company_dict = copy.deepcopy(data_address_company_dict_backup)

In [15]:
#checking if the first 6 letters of the address fulfils the anonymzation or not
#this means there is at least k number of data which has the same address
data_address_company_dict_1 = make_not_anonymized_list_initial(data_address_company_dict,data_address_company,20)

0


In [16]:
#checking if the first 6 letters of the address fulfils the anonymzation or not
#this means there is at least k number of data which has the same address
for i in range(1,10):
    start = time.time()
    if i == 1:
        data_address_company_dict_start = copy.deepcopy(data_address_company_dict_1)
    else:
        data_address_company_dict_start = copy.deepcopy(data_address_company_anonymized_dict_final)
    group_list_tmp = make_group_list(data_address_company_dict_start,i)
    not_anonymized_and_group_list = make_not_anonymized_and_group_list(data_address_company_dict_start,group_list_tmp,20)
    anonymized_group_list,data_address_company_anonymized_dict_final = make_anonymized_group_list(not_anonymized_and_group_list[0],not_anonymized_and_group_list[1],data_address_company_dict_start,20,i)
    data_address_company_anonymized_dict_final = make_anonymized_data_dict(anonymized_group_list,data_address_company_anonymized_dict_final,64,i)
    print(time.time()-start)

2.785268545150757
4.377633810043335
6.554855823516846
5.783883810043335
5.146415948867798
3.3385281562805176
3.1439290046691895
2.9404962062835693
3.815218687057495


### validation for each addres_float that there is at least k number of data with the same float value

In [17]:
result = [[data_address_company_anonymized_dict_final[i]["address_float_{}".format(j)] for i in range(len(data_address_company_anonymized_dict_final))]for j in range(10)]

In [40]:
Counter(result[0]).most_common()[-5:]

[('0111111111011011100110000101101011011000010110011101110101011011', 110),
 ('0111111111011011010110100101110101011100100110000101110011011010', 98),
 ('0111111111011011010110100101110101011100100110000101100111011010', 93),
 ('0111111111011010110110111101110101011110100110000101100111011010', 82),
 ('0111111111011011010110100101101110011000010110110101101001011010', 66)]

In [41]:
Counter(result[1]).most_common()[-5:]

[('0111111111011100110110100001101001011011010110111101110000000000', 21),
 ('0111111111011010000110100101101111011000000000000000000000000000', 20),
 ('0111111111011010000110100101111001011011110111001101101000011001', 20),
 ('0111111111011100100110000101110011011010000110100101111010011100', 20),
 ('0111111111011100110110100001101001011011100000000000000000000000', 20)]

In [42]:
Counter(result[2]).most_common()[-5:]

[('0111111111011100110110111101110101011010110111010101110100011100', 21),
 ('0111111111011010010110111101101011011000010000000000000000000000', 20),
 ('0111111111011000010110100101101101011011110110101101110101010111', 20),
 ('0111111111011000010110110101101001001100000000000000000000000000', 20),
 ('0111111111011011110110111001101011011101010110011101100101011011', 20)]

In [43]:
Counter(result[3]).most_common()[-5:]

[('0111111111011010010110101101110101011010000110000101101101011001', 20),
 ('0111111111011000010110101101110101011010110110100101100000000000', 20),
 ('0111111111011000010110101101110101011010000000000000000000000000', 20),
 ('0111111111011110100110000100110000000000000000000000000000000000', 20),
 ('0111111111011011100111010101101101011000010011000000000000000000', 20)]

In [44]:
Counter(result[4]).most_common()[-5:]

[('0111111111011101010111010101110010011000010011000000000000000000', 21),
 ('0111111111011101000110000000000000000000000000000000000000000000', 21),
 ('0111111111011000010111001001101001011101000110000100110000000000', 20),
 ('0111111111011010010110111001101111011101110110000101101101011001', 20),
 ('0111111111011000110110100001101001011110010111010101101101011000', 20)]

In [45]:
Counter(result[5]).most_common()[-5:]

[('0111111111011011110111010100110000000000000000000000000000000000', 21),
 ('0111111111011010010110110101100001011000110110100001101001011001', 21),
 ('0111111111011000010110001101101000011010010011010000000000000000', 20),
 ('0111111111011010110110100101100011011010000111010101110101011010', 20),
 ('0111111111011011110111010001101111000000000000000000000000000000', 20)]

In [46]:
Counter(result[6]).most_common()[-5:]

[('0111111111011000010000000000000000000000000000000000000000000000', 41),
 ('0111111111011000110110100001101001001100000000000000000000000000', 32),
 ('0111111111011000110110100001101001000000000000000000000000000000', 25),
 ('0111111111001000000000000000000000000000000000000000000000000000', 21),
 ('0111111111011011110111010100110000000000000000000000000000000000', 20)]

In [47]:
Counter(result[7]).most_common()

[('0000000000000000000000000000000000000000000000000000000000000000', 20000)]

In [48]:
Counter(result[8]).most_common()

[('0000000000000000000000000000000000000000000000000000000000000000', 20000)]

In [49]:
Counter(result[9]).most_common()

[('0000000000000000000000000000000000000000000000000000000000000000', 20000)]

## float representaiton back to the string representation for validation

In [28]:
#changing the float representation to a numeric value
address_float_value_anonymized = []
for i in range(10):
    tmp_list = [data_address_company_anonymized_dict_final[j]["address_float_{}".format(i)] for j in range(len(data_address_company_anonymized_dict_final))]
    tmp = []
    for j in tmp_list:
        tmp.append(float_to_num(j))
    address_float_value_anonymized.append(tmp)

In [29]:
#changing the float number back into the int value and to the binary representation
address_int_anonymized = []
for j in range(len(address_float_value_anonymized[0])):
    for x,i in enumerate(address_float_value_anonymized):
        if x == 0:
            tmp = bin(int(i[j]/np.float64(1.7976931348623157e+308)*np.float64(2**48-1))).replace("b","").rjust(48,"0")
        else:
            tmp += bin(int(i[j]/np.float64(1.7976931348623157e+308)*np.float64(2**48-1))).replace("b","").rjust(48,"0")
            #print(tmp)
    address_int_anonymized.append(tmp)

In [30]:
#changing the binary representation back to the string representaion
address_anonymized = []
for i in address_int_anonymized:
    for j in range(0,len(i),8):
        if j == 0:
            tmp = chr(int(i[j:j+8],2))
        else:
            if chr(int(i[j:j+8],2)) != "ÿ":
                tmp+=chr(int(i[j:j+8],2))
    address_anonymized.append(tmp[:tmp.find("\x00")])

### validation that there is at least k number of raws which has the same address 

In [50]:
Counter(address_anonymized).most_common()[-5:]

[('ashigarashimogunhakonemachiyumotn', 20),
 ('kamakurashizaimokuza/', 20),
 ('yamatoshifukami/', 20),
 ('hatanoshim', 20),
 ('fujisawashihonkugenuma/', 20)]

In [31]:
#making the dataframe which has the raw address and the anonymized address
data_anonymized_final = pd.DataFrame(np.array([data_address_company_anonymized_dict_final[i]["address_english"] for i in range(len(data_address_company_anonymized_dict_final))]),columns=["raw"])
data_anonymized_final["address_english_anonymized"] = address_anonymized
#shows randomly chosed 20 raws of the dataframe
data_anonymized_final.sample(20)

,raw,address_english_anonymized
15865,yokohamashitsurumikuushiodamachi4choume150banc...,yokohamashitsurumikuushiodak
2955,ayaseshitadekawa2choume12ban28gou,ayaseshi_
18522,hiratsukashiminamikaneme732banchino4,hiratsukashimg
831,odawarashinishisakou1choume3ban8gou,odawarashin_
3960,yokosukashiurasatomachi4choume8−3,yokosukashis
4705,kawasakishikawasakikuasada2choume4ban11gou,kawasakishikawasakikuasa_
5586,kawasakishinakaharakuidanakaﾉmachi33ban5−203gou,kawasakishinakaharakui_
18402,hiratsukashiokazaki4053banchi,hiratsukashiog
9842,yokohamashikanazawakuteramae1choume9ban11gou,yokohamashikanazawakut_
2682,ebinashikokubunkita2choume18ban56gou,ebinashikokubum
